## Capstone Project : Mahindra First Choice  [ EDA Only ]


### Problem Statement
Mahindra First Choice Services (MFCS) is a company of Mahindra Group and is India’s leading chain of multi-brand car workshops with over 335+ workshops present in 267+ towns & 24 states. It has serviced over 10,50,000 cars. The company aims to establish countrywide network of over 400 workshops by March 2018.
Mahindra would now like to leverage the data that they have and address the key issues they have. Read along to know how you can help them improve their business.
The dataset consist of three aspects:
* Customer data: where the details of the customer like the car owned, state and place of residence, order type, etc are present. Data dimension is of 534000 Customer entries
* Invoice data: where information related to customer visits and transactions are recorded, whether a customer as insurance claims, bifurcation of the amount 
 paid, for what type of service did the customer came for, etc…
* Material Inventory: where information related to what kind of service did the customer took and what kind of material was used to service, Labor information and the cost for the service, Plant and plant name where the customer took the service.


### Objective

### Geolocation Based Customer Analysis:
The idea is to explore how various factors like car make & model, time and type of service etc. vary with location. Since the servicing industry is local in nature, this kind of an analysis could possibly render some really interesting business insights.
Furthermore, this analysis will enable us to formulate more concrete machine learning problems.
From the data at hand it is possible to extract insights about customer behaviour especially the following questions can be addressed

#### Problem Statement-1
Identifying the ownership pattern of cars throughout the country. This also captures the problem wherein information regarding the spending patterns can be identified Expected Business Outcome: Mahindra First Choice Services will be benefited in multiple ways. Knowing the ownership pattern targeted marketing campaigns could be carried out. Knowing the spending patterns services could be suited to the particular spending pattern.

#### Problem Statement-2
Identify the type of order each state receives and present it as an interactive visualization. Expected Business Outcome: This could potentially give information about how Mahindra First Choice needs to be prepared to tackle various seasonal cases

### Market Segmentation:
Market segmentation is the process of dividing a market of potential customers into internally homogeneous and mutually heterogeneous groups or segments, based on different characteristics captured in the data. Groups created through such a segmentation exercise many times reveal behavioral patterns which are different from generally accepted segments by the business. The exercise is broadly known as “clustering” and is aimed at finding the consumers who will respond similarly to various stimuli by detecting underlying behavior patterns.
Though clustering falls under a Machine Learning problem category called unsupervised learning, which requires extensive efforts, it is possible to carry out a visual analysis in a relatively short timespan.

#### Problem Statement: 
Customer Lifetime value prediction - Based on Customer segments, predict the revenue that can be extracted from each segment over a life of the car -Regression/Time Series.


## Start of Notebook

#### Why are we doing this
- To Help Mahindra improve its business from analysing the data given like max. car model coming for repair location-wise, top sales garage etc
- To Identify more profitable area or expansion in other regions

#### Stakeholders:
- Marketing Head of Mahindra First [For Marketing Campaign]
- Operational Head/State Head of Mahindra First [For knowing In-Outs operation in Garage]
- Project Manager [i.e. Our Consulting Firm Manager - To which Mahindra First Company are first point of contact with for Analysis]

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import plotly
import plotly.graph_objects as go
import re
import pgeocode

In [2]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.express as px


username = "dashang" 
api_key="bzWQDD1yyqJt1CR6m5un" 
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
plotly.offline.init_notebook_mode(connected=True)

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
# If DataSet lies in another Directory
#os.getcwd()
os.chdir("C:\\Users\\Dashang\\Documents\\GreyAtom_Capstone_MahindraFirst\\") 

In [5]:
customer = pd.read_csv("customer_cleaned_decoded.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning:

Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.



In [6]:
def ReverseEncodingForCustomerData(df):
    df['partner_type']=df['partner_type'].astype(int)
    df['partner_type_decoded'] = df['partner_type'].replace({1:'Retail',2:'Corporate',3:'Fleet',4:'Employee',9001:'Insurance Company',9002:'Surveyor',9003:'Contact Person'})
    
    df['data_origin_decoded'] = df['data_origin'].replace({
                                                                        "Z001":"Camp - Outdoor",
                                                                        "Z002":"Camp - Workshop",
                                                                        "Z003":"Emailers",
                                                                        "Z004":"Fleet",
                                                                        "Z005":"Reference - Customer",
                                                                        "Z006":"Reference - Employee",
                                                                        "Z007":"Reference - Used car dealer",
                                                                        "Z008":"Just Dial/Other helpline",
                                                                        "Z009":"Snapdeal/Other websites",
                                                                        "Z010":"Company website",
                                                                        "Z011":"Float activity",
                                                                        "Z012":"Petrol pump activity",
                                                                        "Z013":"Hoardings/Outdoor Advertisements",
                                                                        "Z014":"Insurance Company",
                                                                        "Z015":"Television AD",
                                                                        "Z016":"Newspaper AD",
                                                                        "Z017":"Newspaper leaflet",
                                                                        "Z018":"Outdoor Sales Activity",
                                                                        "Z019":"Spotted the outlet",
                                                                        "Z020":"Mahindra Sister concern Employee",
                                                                        "Z021":"Other outdoor activity",
                                                                        "Z022":"Radio"
                                                                         })
    
    return df

In [7]:
customer.head()

,business_partner,customer_no,partner_type,data_origin,partner_type_decoded,data_origin_decoded
0,31,1,1,Z001,Retail,Camp - Outdoor
1,32,2,1,Z005,Retail,Reference - Customer
2,34,4,1,Z005,Retail,Reference - Customer
3,35,5,1,Z005,Retail,Reference - Customer
4,42,7,1,Z008,Retail,Just Dial/Other helpline


In [8]:
# customer = ReverseEncodingForCustomerData(customer)

In [9]:
customer.head()

,business_partner,customer_no,partner_type,data_origin,partner_type_decoded,data_origin_decoded
0,31,1,1,Z001,Retail,Camp - Outdoor
1,32,2,1,Z005,Retail,Reference - Customer
2,34,4,1,Z005,Retail,Reference - Customer
3,35,5,1,Z005,Retail,Reference - Customer
4,42,7,1,Z008,Retail,Just Dial/Other helpline


customer.to_csv('customer_cleaned_decoded.csv', index=False)

# Customer EDA

### 1) By which Source Customer came/referred ? 

In [10]:
source = customer.data_origin_decoded.value_counts()

In [11]:
source.index

Index(['Reference - Customer', 'Reference - Employee', 'Camp - Outdoor', 'Camp - Workshop', 'Fleet', 'Spotted the outlet', 'Other outdoor activity', 'Outdoor Sales Activity', 'Hoardings/Outdoor Advertisements', 'Just Dial/Other helpline', 'Newspaper AD', 'Float activity', 'Company website', 'Newspaper leaflet', 'Emailers', 'Insurance Company', 'Petrol pump activity', 'Reference - Used car dealer', 'Television AD', 'Snapdeal/Other websites', 'Mahindra Sister concern Employee', 'Radio'], dtype='object')

In [12]:
fig = go.Figure(data=go.Bar(y=customer.data_origin.value_counts(),x=source.index))
#fig.update_layout(autosize=False,height=1200,width=1200)

fig.update_layout(
    #autosize=False,
    #width=500,
    #height=500,
    title_text='Customer Referral Sources'
)

#fig.update_yaxes(automargin=True)

fig.show()

In [13]:
py.plot(fig,filename="Customer Referral Sources", auto_open=False)

'https://plotly.com/~dashang/1/'

## Garage Data as per Labour Requirement

In [14]:
customer = pd.read_csv("customer_cleaned_decoded.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning:

Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.



In [15]:
 customer.partner_type_decoded.value_counts()

Retail               555045
Corporate                68
Fleet                    62
Insurance Company        57
Contact Person           56
Employee                 48
Surveyor                  2
Name: partner_type_decoded, dtype: int64

In [16]:
source = customer.partner_type_decoded.value_counts()

In [17]:
plotly.offline.init_notebook_mode(connected=True)

In [18]:
fig = go.Figure(data=go.Bar(y=customer.partner_type_decoded.value_counts(),x=source.index,text=customer.partner_type_decoded.value_counts(),
            textposition='auto'))
#fig.update_layout(autosize=False,height=1200,width=1200)

fig.update_layout(
    #autosize=False,
    #width=500,
    #height=500,
    title_text='Customer Partner Type'
)

#fig.update_yaxes(automargin=True)

fig.show()

## All India Most widely Serviced Company Model

In [19]:
top5_All_India_Widely_serviced_models = invoice.model.value_counts()[:10]

NameError: name 'invoice' is not defined

In [ ]:
top5_All_India_Widely_serviced_models[0]

In [ ]:
fig = go.Figure(data=go.Bar(y=top5_All_India_Widely_serviced_models,x=top5_All_India_Widely_serviced_models.index,text=top5_All_India_Widely_serviced_models,
            textposition='auto'))
#fig.update_layout(autosize=False,height=1200,width=1200)

fig.update_layout(
    #autosize=False,
    #width=500,
    #height=500,
    title_text='All India Most widely Serviced Company Model'
)

#fig.update_yaxes(automargin=True)

fig.show()

In [ ]:
py.plot(fig,filename="All India Most widely Serviced Company Model", auto_open=False)

## All India Least widely Serviced Company Model

In [ ]:
top5_All_India_Widely_serviced_models = invoice.model.value_counts()[-10:]

In [ ]:
top5_All_India_Widely_serviced_models[0]

In [ ]:
fig = go.Figure(data=go.Bar(y=top5_All_India_Widely_serviced_models,x=top5_All_India_Widely_serviced_models.index,text=top5_All_India_Widely_serviced_models,
            textposition='auto'))
#fig.update_layout(autosize=False,height=1200,width=1200)

fig.update_layout(
    #autosize=False,
    #width=500,
    #height=500,
    title_text='All India Least widely Serviced Company Model'
)

#fig.update_yaxes(automargin=True)

fig.show()

In [ ]:
py.plot(fig,filename="All India Least widely Serviced Company Model", auto_open=False)

## Most used Car type

## Group by Most company Car Type and location

In [ ]:
invoice = pd.read_csv("invoice_model_type_zone_servicetime.csv")

In [ ]:
invoice.head()

In [ ]:
invoice.groupby(['state']).invoice_no.count()

In [ ]:
grp=invoice.groupby(['model_type', 'state']).invoice_no.count()   #, as_index=False).count().sort_values(['Major', 'GPA'])

In [ ]:
grp_loc=invoice[['model','state','invoice_no']]

In [ ]:
grp_loc.group

In [ ]:
col_grp=invoice[['model_type','state','invoice_no']].groupby(['model_type'])['state'] #['state'].count().sort_values(ascending=False)

In [ ]:
col_grp.value_counts()


In [ ]:
import plotly.express as px
#df = px.data.tips()
fig = px.sunburst(invoice, path=['state', 'model_type'], values='total_amt')
fig.show()

In [ ]:
py.plot(fig,filename="State ModelType Sunburn", auto_open=False)

### Time Series Based Graph

In [ ]:
t=  pd.read_csv("invoice_model_type_zone_servicetime.csv")

In [ ]:
t.dtypes

In [ ]:
t['invoice_datetime'] = pd.to_datetime(t.invoice_datetime) # t['invoice_datetime'].dt.strftime('%Y-%m')

In [ ]:
t['invoice_displayDate'] = t['invoice_datetime'].dt.strftime('%Y-%m')

In [ ]:
t.head()

In [ ]:
df = t[['invoice_displayDate','state','total_amt']]

In [ ]:
df_grp=t[['invoice_displayDate','state','total_amt']].groupby(['state'])['total_amt','invoice_displayDate']

In [ ]:
df_grp.describe()

In [ ]:
df_grp.describe()

In [ ]:
df_grp.value_counts().index

In [ ]:
fig = px.line(df_grp.value_counts(), x="invoice_displayDate", y="total_amt", color="state",line_group="state")
fig.show()

In [ ]:
geo1 = go.Scatter(
x=df['invoice_displayDate'],
y=df['total_amt'],
mode='lines',
marker=dict(color=df['state'],size=4, showscale=False),
name='geo',
showlegend=True)


data = [geo1]

layout = dict(
title='Working VISA in UK by Regions',
xaxis=dict(title='Year'),
yaxis=dict(title='Number'), showlegend=True)
fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
fig = px.line(t, x="invoice_displayDate", y="total_amt", color="state",line_group="state")
fig.show()

In [ ]:
fig = px.bar(t, x="sex", y="total_bill", color='smoker', barmode='group',
             height=400)
fig.show()

In [ ]:
fig = go.Figure([go.Scatter(x=t['invoice_datetime'], y=t['total_amt'], color=t['state'])])
fig.show()

In [ ]:
def get_top_values(dataset,col,min_percentage):
    filtered_df = dataset[col].value_counts()
    filtered_df=filtered_df.reset_index()
    filtered_df[col] = (filtered_df[col]/sum(filtered_df[col]))*100
    filtered_df = filtered_df.sort_values(col)
    filtered_df = filtered_df[filtered_df[col]>min_percentage]
    top_values  = filtered_df['index'].tolist()
    top_values_df = dataset[dataset[col].isin(top_values)]
    return top_values_df, top_values

def plot_TimeSeries(dataset, time_col, cat_col, num_col):
    fig, ax = plt.subplots(figsize=(15,7))
    plt.title("'{}'wise '{}' in top '{}'".format(time_col, num_col, cat_col))
    dataset.groupby([time_col,cat_col])[num_col].sum().unstack().plot(ax=ax,marker='o')

In [ ]:
top_state_df, top_state = get_top_values(data, 'state',1)
plot_TimeSeries(top_state_df,'Year','state','total_amt')

# Map Based Graph

### Map by model

In [20]:
t=  pd.read_csv("invoice_model_type_zone_servicetime.csv")

In [21]:
t.head()

,model,customer_type,customer_no,gate_pass_time,invoice_no,job_card_no,km_reading,labour_total,make,order_type,pin_code,plant_code,plant_name,regn_no,total_amt,user_id,pin_city,pin_district,pin_location,latitude,longitude,state,zones,invoice_datetime,jobcard_datetime,Service_Time,model_type
0,spark,retail,67849,0:00:00,7005200002,168303,49317,1203.14,general motors,paid service,400601,bc01,thane,ka19ma1291,4051.95,bc01fs1,Thane,Maharashtra,"Thane Bazar, Thane H.O, Thane R.S.",19.1941,73.0002,maharashtra,west zone,01/02/15 11:30,12/13/2014 14:29,19 days 21:00:53.000000000,hatchback
1,indica,retail,84419,0:00:00,7005200003,173997,78584,804.26,tata motors,smc value package,400603,bc01,thane,mh43r3046,1001.29,bc01sa2,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.0002,maharashtra,west zone,01/03/15 10:07,01/02/15 14:12,0 days 19:55:14.000000000,hatchback
2,zen,retail,81055,0:00:00,7005200004,173889,33985,180.19,maruti suzuki,running repairs,400607,bc01,thane,ap09ax0582,233.14,bc01sa2,Thane,Maharashtra,"Chitalsar Manpada, Sandozbaugh",19.1941,73.0002,maharashtra,west zone,01/03/15 11:12,01/02/15 11:40,0 days 23:32:13.000000000,hatchback
3,indica,retail,84419,0:00:00,7005200005,174158,78584,0.00,tata motors,smc redemption,400603,bc01,thane,mh43r3046,0.00,bc01sa2,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.0002,maharashtra,west zone,01/03/15 11:40,01/03/15 10:12,0 days 01:28:12.000000000,hatchback
4,santro xing,retail,18980,0:00:00,7005200006,173860,50057,1202.97,hyundai,paid service,400603,bc01,thane,mh04cd9768,2747.12,bc01sa2,Thane,Maharashtra,"Thane East, Kopri Colony",19.1941,73.0002,maharashtra,west zone,01/03/15 12:07,01/02/15 10:45,1 days 01:21:35.000000000,hatchback


In [22]:
t['make'].value_counts()

maruti suzuki           147102
mahindra &  mahindra     87356
hyundai                  85342
tata motors              58034
general motors           29092
ford                     21616
toyota                   19466
honda                    15464
skoda                     6515
fiat                      6437
volkswagen                6150
renault                   3429
nissan                    3090
mitsubishi motors         1427
daewoo                     392
bmw                        257
mercedes benz              256
audi                       220
hindustan motors           219
force                      207
premier¬†automobiles        93
sonalika                    87
land rover                  21
volvo                       19
san motors                   8
jeep                         7
porche                       6
morris                       2
Name: make, dtype: int64

In [23]:
t['make'].value_counts()[-7:]

sonalika      87
land rover    21
volvo         19
san motors     8
jeep           7
porche         6
morris         2
Name: make, dtype: int64

In [24]:
model_types = list(t['make'].value_counts()[-7:].index)
model_types = [
#     'maruti suzuki',
#  'mahindra &  mahindra',
#  'hyundai',
#  'tata motors',
#  'general motors',
#  'ford',
#  'toyota',
#  'honda',
#  'skoda',
#  'fiat',
#  'volkswagen',
 'sonalika',
 'land rover',
 'volvo',
 'san motors',
  'jeep',
 'porche',
 'morris' 
              ]

In [25]:
"A"+","+str(23)

'A,23'

In [28]:
data = []
for m in model_types:
    model_data = dict(
            lat = t.loc[t['make'] == m,'latitude'],
            lon = t.loc[t['make'] == m,'longitude'],
            name = m+","+str(23),
            marker = dict(size = 8, opacity = 0.5),
            type = 'scattermapbox'
        )
    data.append(model_data)

mapbox_access_token = 'pk.eyJ1IjoiZGFzaGFuZyIsImEiOiJja2JueGE5NmIwYmN2MnBwbnhlbmVra3UzIn0.jmgPp8_lVLbOkxCNUUJiQQ'

layout = dict(
    height = 800,
    margin = dict(t=0, b=0, l=0, r=0),
    font = dict(color='#FFFFFF', size=11),
    paper_bgcolor = '#000000',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=28.644800,
            lon=77.216721
        ),
        pitch=0,
        zoom=3,
        style='dark'
    ),
)


updatemenus=list([

    dict(
        buttons=list([
            dict(
                args=['mapbox.style', 'dark'],
                label='Dark',
                method='relayout'
            ),                    
            dict(
                args=['mapbox.style', 'light'],
                label='Light',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'outdoors'],
                label='Outdoors',
                method='relayout'
            ),
            dict(
                args=['mapbox.style', 'satellite-streets'],
                label='Satellite with Streets',
                method='relayout'
            )                    
        ]),
        direction = 'up',
        x = 0.75,
        xanchor = 'left',
        y = 0.05,
        yanchor = 'bottom',
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    ),    
    
    dict(
         buttons=list([
#             dict(label = 'All Models ',
#                  method = 'update',
#                  args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True,True , True, True, True, True, True]}]),
#             dict(label = 'maruti suzuki',
#                  method = 'update',
#                  args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False,False , False, False, False, False, False]}]),
#             dict(label = 'mahindra &  mahindra',
#                  method = 'update',
#                  args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False,False , False, False, False, False, False]}]),
#              dict(label = 'hyundai',
#                  method = 'update',
#                  args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False,False , False, False, False, False, False]}]),
#              dict(label = 'tata motors',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False,False , False, False, False, False, False]}]),
#              dict(label = 'general motors',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False,False , False, False, False, False, False]}]),
#              dict(label = 'ford',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False,False , False, False, False, False, False]}]),
#              dict(label = 'toyota',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False,False , False, False, False, False, False]}]),
#              dict(label = 'honda',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False,False , False, False, False, False, False]}]),
             
             
#              dict(label = 'skoda',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False,False , False, False, False, False, False]}]),
             
#              dict(label = 'fiat',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False,False , False, False, False, False, False]}]),
             
#              dict(label = 'volkswagen',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False,False , False, False, False, False, False]}]),
             
#              dict(label = 'sonalika',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, False, True, False, False,False , False, False, False, False, False]}]),
             
#              dict(label = 'land rover',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, True, False,False , False, False, False, False, False]}]),
             
#              dict(label = 'volvo',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, True,False , False, False, False, False, False]}]),
             
#              dict(label = 'san motors',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False,True , False, False, False, False, False]}]),
             
#              dict(label = 'jeep',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False,False , True, False, False, False, False]}]),
             
#              dict(label = 'porche',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False,False , False, True, False, False, False]}]),
             
#              dict(label = 'morris',
#                  method = 'update',
#                  args = [{'visible': [False, False, False, False, False, False, False, False, False, False, False, False,False , False, False, True, False, False]}])
          
             
             
             dict(label = 'All Models ',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True ]}]),
             
             dict(label = 'sonalika',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False ]}]),
             
             dict(label = 'land rover',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False]}]),
             
             dict(label = 'volvo',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False]}]),
             
             dict(label = 'san motors',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False]}]),
             
             dict(label = 'jeep',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False]}]),
             
             dict(label = 'porche',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False]}]),
             
             dict(label = 'morris',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True]}])      
             
             
             
        ]),
        direction = 'down',
        x = 0.01,
        xanchor = 'left',
        y = 0.99,
        yanchor = 'bottom',
        bgcolor = '#000000',
        bordercolor = '#FFFFFF',
        font = dict(size=11)
    )
])


annotations = [dict(text='All India Car Make location', 
         font=dict(color='#FFFFFF',size=14), borderpad=10, 
         x=0.05, y=0.05, xref='paper', yref='paper', align='left', showarrow=False, bgcolor='black')]

layout['title'] = 'Make'
layout['updatemenus'] = updatemenus
layout['annotations'] = annotations

In [29]:
figure = dict(data=data, layout=layout)
py.iplot(figure, filename='All-India-Make-dropdown')

### D3 prototype


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import plotly
import plotly.graph_objects as go
import re
import pgeocode

In [ ]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import plotly.express as px


username = "dashang" 
api_key="bzWQDD1yyqJt1CR6m5un" 
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# If DataSet lies in another Directory
#os.getcwd()
os.chdir("C:\\Users\\Dashang\\Documents\\GreyAtom_Capstone_MahindraFirst\\") 

In [ ]:
t=  pd.read_csv("invoice_model_type_zone_servicetime.csv")

In [ ]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import pandas as pd


import plotly.graph_objs as go

t.head()

In [ ]:
app = dash.Dash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])
server = app.server
app.config.suppress_callback_exceptions = True

In [ ]:
#set the app.layout
app.layout = html.Div([
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Tab one', value='tab-1'),
        dcc.Tab(label='Tab two', value='tab-2'),
    ]),
    html.Div(id='tabs-content')
])

In [ ]:
# #callback to control the tab content
# @app.callback(Output('tabs-content', 'children'),
#               [Input('tabs', 'value')])
# def render_content(tab):
#     if tab == 'tab-1':
#         return html.H1('Tab 1')
#     elif tab == 'tab-2':
#         return html.H1('Tab 2')

In [ ]:
@app.callback(Output('tabs-content', 'children'),
              [Input('tabs', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div(dash_table.DataTable(
                            id='table-sorting-filtering',
                            columns=[
                                {'name': i, 'id': i, 'deletable': True} for i in ['state','make'] #t.columns
                            ],
                            style_table={'overflowX': 'scroll'},
                            style_cell={
                                'height': '90',
                                # all three widths are needed
                                'minWidth': '140px', 'width': '140px', 'maxWidth': '140px',
                                'whiteSpace': 'normal'
                            },
                            page_current= 0,
                            page_size= 50,
                            page_action='custom',
                            filter_action='custom',
                            filter_query='',
                            sort_action='custom',
                            sort_mode='multi',
                            sort_by=[]
                        )
                        )
    elif tab == 'tab-2':
        return html.Div([
            dcc.Graph(
            id='rating-price',
            figure={
                'data': [
                    dict(y = t['total_amt'],x = t['pin_city'],mode ='markers', opacity = 0.7,
                        marker = {'size': 8,'line': {'width': 0.5, 'color': 'white'}},name = 'total_amt v city'
                        ) 
                        ],
                'layout': dict(xaxis = {'type': 'log', 'title': 'City'},yaxis = {'title': 'Total Amount'},
                        margin = {'l': 40, 'b': 40, 't': 10, 'r': 10},
                        legend = {'x': 0, 'y': 1},
                        hovermode = 'closest'
                        )
                    }
            )
            ])

In [ ]:
operators = [['ge ', '>='],
             ['le ', '<='],
             ['lt ', '<'],
             ['gt ', '>'],
             ['ne ', '!='],
             ['eq ', '='],
             ['contains '],
             ['datestartswith ']]
def split_filter_part(filter_part):
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]
                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try:
                        value = float(value_part)
                    except ValueError:
                        value = value_part
    # word operators need spaces after them in the filter string,
                # but we don't want these later
                return name, operator_type[0].strip(), value
    return [None] * 3
@app.callback(Output('table-sorting-filtering', 'data'),
[Input('table-sorting-filtering', "page_current"),
Input('table-sorting-filtering', "page_size"),
Input('table-sorting-filtering', 'sort_by'),
Input('table-sorting-filtering', 'filter_query')])
def update_table(page_current, page_size, sort_by, filter):
    filtering_expressions = filter.split(' && ')
    dff = t
    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)
        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
        # these operators match pandas series operator method names
            dff = dff.loc[getattr(dff[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]
        elif operator == 'datestartswith':
        # this is a simplification of the front-end filtering logic,
        # only works with complete fields in standard format
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]
    if len(sort_by):
        dff = dff.sort_values([col['column_id'] for col in sort_by],ascending=[
            col['direction'] == 'asc'
            for col in sort_by],
            inplace=False
            )
    page = page_current
    size = page_size
    return dff.iloc[page * size: (page + 1) * size].to_dict('records')

In [ ]:
if __name__ == "__main__":
    app.run_server()